In [17]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [18]:
def extract_balance_sheet(company_name):
    url = f"https://www.screener.in/company/{company_name}/#balance-sheet"

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    try:
        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.ID, "balance-sheet")))
        balance_sheet_section = driver.find_element(By.ID, "balance-sheet")
        driver.execute_script("arguments[0].scrollIntoView();", balance_sheet_section)
        time.sleep(5) 
        table = driver.find_element(By.XPATH, "//section[@id='balance-sheet']//table[contains(@class, 'data-table')]")
        rows = table.find_elements(By.TAG_NAME, "tr")
        balance_sheet_data = {}
        header_cells = rows[0].find_elements(By.XPATH, ".//th")
        years = [cell.text.strip() for cell in header_cells if cell.text.strip()]
        
        if not years:
            return None

        for row in rows[1:]:  
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) > 1:
                row_name = cells[0].text.strip()  
                row_values = [cell.text.strip().replace(",", "") for cell in cells[1:]]

                if row_name:
                    balance_sheet_data[row_name] = dict(zip(years, row_values))

        driver.quit()
        balance_sheet_df = pd.DataFrame.from_dict(balance_sheet_data, orient='index', columns=years)
        print(balance_sheet_df)

        return balance_sheet_data, balance_sheet_df

    except Exception as e:
        print(f"Error occurred: {e}")
        driver.quit()
        return None, None



In [19]:
# Example usage
company = "KRISHCA"  # Example company name
data_dict, data_df = extract_balance_sheet(company)



                    Mar 2020 Mar 2021 Mar 2022 Mar 2023 Mar 2024 Sep 2024
Equity Capital             3        3        5        9       12       15
Reserves                  -2       -4       -3        2       27       95
Borrowings +               8       13        9       12       20       32
Other Liabilities +        0        1        4       12       18       24
Total Liabilities          9       13       15       35       78      166
Fixed Assets +             6        6        4        4        5       26
CWIP                       0        0        1        5       20        5
Investments                0        0        0        0        0        1
Other Assets +             4        7       10       26       52      135
Total Assets               9       13       15       35       78      166


DICTIONARY FORMAT ORGANIZATION

In [21]:
data_df

,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Mar 2024,Sep 2024
Equity Capital,3,3,5,9,12,15
Reserves,-2,-4,-3,2,27,95
Borrowings +,8,13,9,12,20,32
Other Liabilities +,0,1,4,12,18,24
Total Liabilities,9,13,15,35,78,166
Fixed Assets +,6,6,4,4,5,26
CWIP,0,0,1,5,20,5
Investments,0,0,0,0,0,1
Other Assets +,4,7,10,26,52,135
Total Assets,9,13,15,35,78,166


In [22]:
data_dict

{'Equity Capital': {'Mar 2020': '3',
  'Mar 2021': '3',
  'Mar 2022': '5',
  'Mar 2023': '9',
  'Mar 2024': '12',
  'Sep 2024': '15'},
 'Reserves': {'Mar 2020': '-2',
  'Mar 2021': '-4',
  'Mar 2022': '-3',
  'Mar 2023': '2',
  'Mar 2024': '27',
  'Sep 2024': '95'},
 'Borrowings +': {'Mar 2020': '8',
  'Mar 2021': '13',
  'Mar 2022': '9',
  'Mar 2023': '12',
  'Mar 2024': '20',
  'Sep 2024': '32'},
 'Other Liabilities +': {'Mar 2020': '0',
  'Mar 2021': '1',
  'Mar 2022': '4',
  'Mar 2023': '12',
  'Mar 2024': '18',
  'Sep 2024': '24'},
 'Total Liabilities': {'Mar 2020': '9',
  'Mar 2021': '13',
  'Mar 2022': '15',
  'Mar 2023': '35',
  'Mar 2024': '78',
  'Sep 2024': '166'},
 'Fixed Assets +': {'Mar 2020': '6',
  'Mar 2021': '6',
  'Mar 2022': '4',
  'Mar 2023': '4',
  'Mar 2024': '5',
  'Sep 2024': '26'},
 'CWIP': {'Mar 2020': '0',
  'Mar 2021': '0',
  'Mar 2022': '1',
  'Mar 2023': '5',
  'Mar 2024': '20',
  'Sep 2024': '5'},
 'Investments': {'Mar 2020': '0',
  'Mar 2021': '0',
  'M